<a href="https://colab.research.google.com/github/t868872/motion_simulator/blob/main/gravity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
取説
https://docs.google.com/document/d/1LEAl-72Xk02voOQ76DgfbS76XWdfX2Vjdy7r1LQadwo/edit?usp=sharing
"""

from IPython.display import HTML

import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.animation import FuncAnimation

import numpy as np
%matplotlib nbagg

rc('animation', html='jshtml')


class Matter:
    #物体　プロパティが入る
    #星を作る
    def __init__(self, name, m=0.0, c=None, v=None, color=None):
        self.Property = {}
        self.Property["Name"] = name
        self.Property["Mass"] = np.float64(m)
        self.Property["Coordinate"] = np.float64(c) if c is not None else [np.float64(0.0), np.float64(0.0)]
        self.Property["Velocity"] = np.float64(v) if v is not None else [np.float64(0.0), np.float64(0.0)]
        self.Property["Color"] = color if color is not None else "blue"

    def __repr__(self):
        return f"Matter{self.Property}"

    def GetProperty(self, key):
        return self.Property[key]

    def SetProperty(self, key, value):
        self.Property[key] = value
        return value


class Grav_Field:
    #これに入れたものをすべて計算させる
    def __init__(self, grav_calc, *matter, interval=0.1):
        self.Time = [np.float64(0)]
        self.Step = 0
        self.Grav_calc = grav_calc
        self.Matters = list()
        self.Loci = list()
        self.Velo_History = list()
        self.Interval = np.float64(interval)
        for M in matter:
            if not(type(M) is Matter):
                raise ValueError("Arguments after first one (except interval) must be Matter.")
            Coordinate = M.GetProperty("Coordinate")
            Velocity = M.GetProperty("Velocity")
            self.Matters.append(M)
            self.Loci.append([[Coordinate[0]], [Coordinate[1]]])
            self.Velo_History.append([[Velocity[0]],[Velocity[1]],[np.linalg.norm(Velocity)]])

        """
        Time 時間の変数
        Step 計算した回数
        Grav_calc 重力を計算する関数　二体の質量のlist,二体間の距離を変数にとる
        Matters 計算する物体全部のlist 要素はMatter
        Loci 物体ごとに軌跡を保存する Loci[i][j][k]はstep=kのときのi番目の物体の座標のj(=0,1)番目の成分
        Velo_History 速度の履歴[0]:x,[1]:y,[2]:||
        Interval 運動方程式を計算する時間間隔
        """

    #見ての通り
    def Change_Interval(self, new_interval):
        self.Interval = new_interval

    #これまでの軌跡を全消し
    def Remove_Loci(self):
        self.Step = 0
        self.Time = [self.Time[-1]]
        for i, Matter in enumerate(self.Matters):
            Coordinate = Matter.GetProperty("Coordinate")
            Velocity = Matter.GetProperty("Velocity")
            self.Loci.append([[Coordinate[0]], [Coordinate[1]]])
            self.Velo_History.append([[Velocity[0]],[Velocity[1]],[np.linalg.norm(Velocity)]])  # ← 履歴を消して今の位置だけにする

    #interval秒後の世界へ
    def Next(self, ifdisplay = False):
        if ifdisplay:
            ret = [self.Time[-1]]
        else:
            ret = None

        def dir(vector):
            Size = np.linalg.norm(vector)
            return [vector[0] / Size, vector[1] / Size] #方向が同じ単位ベクトルを返す
        #各物体に対して実行する
        for i, Matter1 in enumerate(self.Matters):
            Coordinate1 = Matter1.GetProperty('Coordinate')
            Mass1 = Matter1.GetProperty('Mass')
            #重力の和を計算(grav)
            grav = [0.0, 0.0]
            for j, Matter2 in enumerate(self.Matters):
                if i == j:
                    continue

                Coordinate = Matter2.GetProperty("Coordinate")
                Mass2 = Matter2.GetProperty("Mass")

                rplace = [Coordinate[0] - Coordinate1[0], Coordinate[1] - Coordinate1[1]]
                direction = dir(rplace)
                r = np.linalg.norm(rplace)
                r = max(r, np.float64(1e-10)) #ゼロ除算を防いでおく

                tmp_grav = self.Grav_calc([Mass1, Mass2], r)
                grav[0] += tmp_grav * direction[0]
                grav[1] += tmp_grav * direction[1]

            #更新部分
            dt = self.Interval

            Acceralation = [grav[0] / Mass1, grav[1] / Mass1]
            Velocity = Matter1.GetProperty('Velocity')
            Coordinate = Matter1.GetProperty("Coordinate")

            new_Velocity = [Velocity[0] + Acceralation[0]*dt, Velocity[1] + Acceralation[1]*dt]
            new_Coordinate = [Coordinate[0] + new_Velocity[0]*dt, Coordinate[1] + new_Velocity[1]*dt]

            Matter1.SetProperty('Velocity', new_Velocity)
            Matter1.SetProperty('Coordinate', new_Coordinate)
            #ここまで

            self.Loci[i][0].append(new_Coordinate[0])
            self.Loci[i][1].append(new_Coordinate[1])
            self.Velo_History[i][0].append(new_Velocity[0])
            self.Velo_History[i][1].append(new_Velocity[1])
            self.Velo_History[i][2].append(np.linalg.norm(new_Velocity))
            if ifdisplay:
                ret.append(new_Coordinate)
                ret.append(new_Velocity)

        self.Step += 1
        self.Time.append(self.Time[-1] + self.Interval)
        return ret

    #nextをくりかえす
    def Simulate(self, times, ifdisplay = False):
        if ifdisplay:
            txt=["coordinate", "velocity"]
            def txt_add_num(txt_list,num):
                return [v+str(num) for v in txt_list]
            string=["t"]
            for i in range(len(self.Matters)):
                string+=txt_add_num(txt,i)
            print(*string)
        for _ in range(times):
            ret = self.Next(ifdisplay=ifdisplay)
            if ifdisplay:
                print(*ret,sep=", ")

    #描画をする関数
    def Animation(self, subdiv=100, interval=50):
        xmax = -1*float("inf")
        xmin = float("inf")
        ymax = -1*float("inf")
        ymin = float("inf")
        for i, M in enumerate(self.Matters):
            for f in range(self.Step):
                xmax = max(xmax, self.Loci[i][0][f])
                xmin = min(xmin, self.Loci[i][0][f])
                ymax = max(ymax, self.Loci[i][1][f])
                ymin = min(ymin, self.Loci[i][1][f])
        xRange = xmax - xmin
        yRange = ymax - ymin

        vxmax = -1*float("inf")
        vxmin = float("inf")
        vymax = -1*float("inf")
        vymin = float("inf")
        vmax = -1*float("inf")
        vmin = float("inf")
        for i, M in enumerate(self.Matters):
            for f in range(self.Step):
                vxmax = max(vxmax, self.Velo_History[i][0][f])
                vxmin = min(vxmin, self.Velo_History[i][0][f])
                vymax = max(vymax, self.Velo_History[i][1][f])
                vymin = min(vymin, self.Velo_History[i][1][f])
                vmax = max(vmax, self.Velo_History[i][2][f])
                vmin = min(vmin, self.Velo_History[i][2][f])
        vxRange = vxmax - vxmin
        vyRange = vymax - vymin
        vrange = vmax-vmin

        fig, ax = plt.subplots(
            2,
            3,
            figsize = (9, 6)
            )
        fig.subplots_adjust(
            wspace = 0.5,
            hspace = 0.5
        )

        ax[0,0].set_title("y-x graph")
        ax[0,0].set_xlabel("x[m]")
        ax[0,0].set_ylabel("y[m]")
        ax[0,0].set_xlim(min(xmin, ymin) - (max(xmax, ymax)-min(xmin, ymin))*0.1, max(xmax, ymax) + (max(xmax, ymax)-min(xmin, ymin))*0.1)
        ax[0,0].set_ylim(min(xmin, ymin) - (max(xmax, ymax)-min(xmin, ymin))*0.1, max(xmax, ymax) + (max(xmax, ymax)-min(xmin, ymin))*0.1)
        for i, Matter in enumerate(self.Matters):
            ax[0,0].plot(
                self.Loci[i][0],
                self.Loci[i][1],
                "-",
                color = Matter.GetProperty("Color"),
                markersize = 1,
                label = Matter.GetProperty("Name")
                )
        ax[0,0].legend(loc="upper left")

        ax[1,0].set_title("trails")
        ax[1,0].set_xlabel("x[m]")
        ax[1,0].set_ylabel("y[m]")
        ax[1,0].set_xlim(min(xmin, ymin) - (max(xmax, ymax)-min(xmin, ymin))*0.1, max(xmax, ymax) + (max(xmax, ymax)-min(xmin, ymin))*0.1)
        ax[1,0].set_ylim(min(xmin, ymin) - (max(xmax, ymax)-min(xmin, ymin))*0.1, max(xmax, ymax) + (max(xmax, ymax)-min(xmin, ymin))*0.1)

        ax[0,1].set_title("x-t graph")
        ax[0,1].set_xlabel("time[s]")
        ax[0,1].set_ylabel("x[m]")
        ax[0,1].set_xlim(0, self.Time[-1])
        ax[0,1].set_ylim(min(xmin, ymin) - (max(xmax, ymax)-min(xmin, ymin))*0.1, max(xmax, ymax) + (max(xmax, ymax)-min(xmin, ymin))*0.1)

        ax[1,1].set_title("y-t graph")
        ax[1,1].set_xlabel("time[s]")
        ax[1,1].set_ylabel("y[m]")
        ax[1,1].set_xlim(0, self.Time[-1])
        ax[1,1].set_ylim(min(xmin, ymin) - (max(xmax, ymax)-min(xmin, ymin))*0.1, max(xmax, ymax) + (max(xmax, ymax)-min(xmin, ymin))*0.1)

        ax[0,2].set_title("v-t graph")
        ax[0,2].set_xlabel("time[s]")
        ax[0,2].set_ylabel("speed[m/s]")
        ax[0,2].set_xlim(0, self.Time[-1])
        ax[0,2].set_ylim(vmin - vrange*0.1, vmax + vrange*0.1)

        ax[1,2].set_title("Velocity")
        ax[1,2].set_xlabel("Velocity_x[m/s]")
        ax[1,2].set_ylabel("Velocity_y[m/s]")
        ax[1,2].set_xlim(min(vxmin, vymin) - (max(vxmax, vymax)-min(vxmin, vymin))*0.1, max(vxmax, vymax) + (max(vxmax, vymax)-min(vxmin, vymin))*0.1)
        ax[1,2].set_ylim(min(vxmin, vymin) - (max(vxmax, vymax)-min(vxmin, vymin))*0.1, max(vxmax, vymax) + (max(vxmax, vymax)-min(vxmin, vymin))*0.1)

        time_text1 = ax[1,0].text(
            0.02, 0.95, "",
            transform = ax[1,0].transAxes,
            ha = "left", va = "top"
            )
        time_text2 = ax[1,2].text(
            0.02, 0.95, "",
            transform = ax[1,2].transAxes,
            ha = "left", va = "top"
            )
        point = []
        trail = []
        velocity = []
        end_velo = []
        xs = []
        ys = []
        speed = []
        for i, Matter in enumerate(self.Matters):
            pt, = ax[1,0].plot([], [], "o", color=Matter.GetProperty("Color"))
            tr, = ax[1,0].plot([], [], "-", color=Matter.GetProperty("Color"), markersize=0.3)
            ve, = ax[1,2].plot([], [], "-", color=Matter.GetProperty("Color"))
            en, = ax[1,2].plot([], [], "^", color=Matter.GetProperty("Color"))
            X, = ax[0,1].plot([], [], "-", color=Matter.GetProperty("Color"), markersize=0.3)
            Y, = ax[1,1].plot([], [], "-", color=Matter.GetProperty("Color"), markersize=0.3)
            s, = ax[0,2].plot([], [], "-", color=Matter.GetProperty("Color"), markersize=0.3)
            point.append(pt)
            trail.append(tr)
            velocity.append(ve)
            end_velo.append(en)
            xs.append(X)
            ys.append(Y)
            speed.append(s)

        def init():
            for pt, tr, ve, en, X, Y, s in zip(point, trail, velocity, end_velo, xs, ys, speed):
                pt.set_data([], [])
                tr.set_data([], [])
                ve.set_data([], [])
                en.set_data([], [])
                X.set_data([], [])
                Y.set_data([], [])
                s.set_data([], [])
            time_text1.set_text("")
            time_text2.set_text("")
            return point, trail, velocity, end_velo, s, time_text1, time_text2

        def update(frame):
            for i, Matter in enumerate(self.Matters):
                x = self.Loci[i][0][frame]
                y = self.Loci[i][1][frame]
                point[i].set_data([x], [y])
                trail[i].set_data(
                    self.Loci[i][0][:frame+1],
                    self.Loci[i][1][:frame+1]
                    )
                time_text1.set_text(f"t = {self.Time[frame]:.2f}")

                vx=self.Velo_History[i][0][frame]
                vy=self.Velo_History[i][1][frame]
                velocity[i].set_data([0, vx], [0, vy])
                end_velo[i].set_data([vx], [vy])
                time_text2.set_text(f"t = {self.Time[frame]:.2f}")

                xs[i].set_data(
                    self.Time[:frame+1],
                    self.Loci[i][0][:frame+1]
                )
                ys[i].set_data(
                    self.Time[:frame+1],
                    self.Loci[i][1][:frame+1]
                )

                speed[i].set_data(
                    self.Time[:frame+1],
                    self.Velo_History[i][2][:frame+1]
                )
            return (*point, *trail, *velocity, *end_velo, *xs, *ys, *speed, time_text1, time_text2)

        ani = FuncAnimation(
            fig,
            update,
            init_func = init,
            frames = range(0,self.Step,max(self.Step//subdiv,1)),
            blit = True,
            interval = interval
            )

        return ani

def grav_calc(masses, distance):
    m1, m2 = masses
    G = 6.67*(10**-11)
    n = -2
    gravity = G * m1 * m2 * (distance ** n)
    return gravity

